In [2]:
import pandas as pd
import numpy as np
import pickle
df = pd.read_csv("Data/Joules_of_Siam_Data - Dataset_Strech.csv")
df.head()

,Year,Month,Date,Population,Temperature,CPI,GDP,Peak
0,2022,12,01-12-22,66.090000,26.300000,106.470000,924033.3333,27124.7
1,2022,11,01-11-22,66.083333,26.249167,106.470000,924033.3333,27832.7
2,2022,10,01-10-22,66.076667,26.198333,106.470000,924033.3333,27041.9
3,2022,9,01-09-22,66.070000,26.147500,106.470000,849468.6667,27808.0
4,2022,8,01-08-22,66.063333,26.096667,108.503333,849468.6667,28951.8


In [102]:
df[["Population", "Temperature", "CPI", "GDP"]].values

array([[6.60900000e+01, 2.63000000e+01, 1.06470000e+02, 9.24033333e+05],
       [6.60833333e+01, 2.62491667e+01, 1.06470000e+02, 9.24033333e+05],
       [6.60766667e+01, 2.61983333e+01, 1.06470000e+02, 9.24033333e+05],
       ...,
       [6.31675000e+01, 2.65175000e+01, 7.02333333e+01, 4.80343333e+05],
       [6.32083333e+01, 2.64850000e+01, 7.02333333e+01, 4.80343333e+05],
       [6.32491667e+01, 2.64525000e+01, 7.02333333e+01, 4.80343333e+05]])

In [53]:
# Function to convert integer to list.
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]
    

# Function to generated prediction.
def assume_predict(value, percent_list, start_date='2023-01', end_date='2033-01'):
    
    results = list()
    temp = list()
    temp_val = value

    percent_list = as_list(percent_list)

    for percent in percent_list:

        # Divided % per years to per months

        percent_month = percent/12

        for current_date in day:
            temp_val = temp_val + (temp_val * percent_month/100)
            temp.append(temp_val)

        
        temp_val = value
        results.append(temp)
        temp = list()
    return results

In [55]:
temp1 = assume_predict(GDP_value, 2)

In [94]:
# Recent values
GDP_value = 924033.33
Population_value = 66.09
CPI_value = 106.47

file = open("./Data/Dashboard_Data/forecast_temperature.pkl", "rb")
forecast_temperature = pickle.load(file)

In [98]:
forecast_temperature[0:120].shape

(120,)

In [72]:
def prepare_data(GDP_percent=2, Population_percent=0.05, CPI_percent=2, start_date='2023-01', end_date='2033-01'):
    
    start_date = np.array(start_date, dtype='datetime64[M]')
    end_date   = np.array(end_date, dtype='datetime64[M]')

    date_delta = end_date - start_date

    if date_delta.astype(int) < 0:
        raise ValueError('Invalid start_date/end_date')
    
    # Get Date, Month, Year values
    date = np.arange(start_date, end_date, dtype='datetime64[M]') #.astype('datetime64[D]')
    months = date.astype('datetime64[M]').astype(int) % 12 + 1
    year = date.astype('datetime64[Y]').astype(int) + 1970

    date = date.astype(str).tolist()

    # Get Assume values
    GDP_forecast = assume_predict(GDP_value, GDP_percent, start_date, end_date)
    Population_forecast = assume_predict(Population_value, Population_percent, start_date, end_date)
    CPI_forecast = assume_predict(CPI_value, CPI_percent, start_date, end_date)

    # Get Static values
    file = open("./Data/Dashboard_Data/forecast_temperature.pkl", "rb")
    temperature_value = pickle.load(file)

    # Create DataFrame

    features_dataframe = pd.DataFrame([date, year, months, Population_forecast[0], temperature_value, 
                         CPI_forecast[0], GDP_forecast[0]]).T
    features_dataframe.columns = ["Date", "Year", "Month", "Population", "Temperature", 
                                  "CPI", "GDP"]
    
    
    return features_dataframe
    

In [77]:
features = prepare_data(GDP_percent=2)

In [80]:
from xgboost import XGBRegressor
XGBoost_model = XGBRegressor()


In [39]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
import pickle

class forecasting():

    def __init__(self, 
                 GDP_percent=2, 
                 Population_percent=0.05, 
                 CPI_percent=2,
                 Start_date = '2023-01',
                 End_date = '2033-01'
                 ):
        
        # Check date eligibility
        self.Check_date_eligibility(Start_date, End_date)

        # Get Static temperature values
        file = open("./Data/Dashboard_Data/forecast_temperature.pkl", "rb")
        Temperature_forecast = pickle.load(file)
        self.Temperature_forecast = Temperature_forecast[0:len(self.Date)]

        # Get forecasted LSTM values
        LSTM_forecast = pickle.load(open("./Data/Dashboard_Data/forecast_LSTM.pkl", "rb"))
        self.LSTM_forecast = LSTM_forecast[0:len(self.Date)]

        # Get forecasted ARIMA values
        ARIMA_forecast = pickle.load(open("./Data/Dashboard_Data/forecast_ARIMA.pkl", "rb"))
        self.ARIMA_forecast = ARIMA_forecast[0:len(self.Date)]

        # Initial values for generating features
        self.GDP_value = 924033.33
        self.Population_value = 66.09
        self.CPI_value = 106.47

        # Initial percentages
        self.GDP_percent = GDP_percent
        self.Population_percent = Population_percent
        self.CPI_percent = CPI_percent

        # Initial XGBoost model
        self.XGBoost_model = XGBRegressor()
        self.XGBoost_model.load_model("./Models/XGBoost_model.json")

        # Initial ANN model
        self.ANN_scaler = pickle.load(open('./Data/monthly_scaler.pkl', 'rb'))
        self.ANN_model = pickle.load(open('./Data/ANN_monthly.pkl', 'rb'))

        # Prepare data
        self.prepare_data()

        # Prediction
        self.get_prediction()

        # Ploting Data
        self.plotting_data()
    
    def Check_date_eligibility(self, start_date, end_date):
        
        start_date = np.array(start_date, dtype='datetime64[M]')
        end_date   = np.array(end_date, dtype='datetime64[M]')

        date_delta = end_date - start_date

        if date_delta.astype(int) < 0:
            raise ValueError('Invalid start_date/end_date')
        else:
            self.Start_date = start_date
            self.End_date   = end_date
            Date        = np.arange(start_date, end_date, dtype='datetime64[M]')
            self.months = Date.astype('datetime64[M]').astype(int) % 12 + 1
            self.years  = Date.astype('datetime64[Y]').astype(int) + 1970
            self.Date   = Date.astype(str).tolist()

    # Function to convert integer to list.
    def as_list(self, x):
        if type(x) is list:
            return x
        else:
            return [x]
    

    # Function to generated prediction.
    def assume_predict(self, value, percent_list):
        
        results = list()
        temp = list()
        temp_val = value

        percent_list = self.as_list(percent_list)

        for percent in percent_list:

            # Divided % per years to per months

            percent_month = percent/12

            for current_date in self.Date:
                temp_val = temp_val + (temp_val * percent_month/100)
                temp.append(temp_val)

            temp_val = value
            results.append(temp)
            temp = list()
        return results[0]
    
    def prepare_data(self):

        # Get Assume values
        GDP_forecast = self.assume_predict(self.GDP_value, self.GDP_percent)
        Population_forecast = self.assume_predict(self.Population_value, self.Population_percent)
        CPI_forecast = self.assume_predict(self.CPI_value, self.CPI_percent)

        # Create DataFrame/features for XGBoost
        features_dataframe = pd.DataFrame([self.Date, self.years, self.months, 
                                           Population_forecast, self.Temperature_forecast, 
                                            CPI_forecast, GDP_forecast]).T
        features_dataframe.columns = ["Date", "Year", "Month", "Population", "Temperature", 
                                    "CPI", "GDP"]
        
        # Convert DataFrame types
        features_dataframe['Date']= pd.to_datetime(features_dataframe['Date'])
        features_dataframe['Year']= features_dataframe['Date'].dt.year
        features_dataframe['Month']= features_dataframe['Date'].dt.month
        
        features_dataframe[["Population", "Temperature", "CPI", "GDP"]] = \
        features_dataframe[["Population", "Temperature", "CPI", "GDP"]].astype(float)

        
        features_dataframe.set_index("Date", inplace=True)
        
        self.XGBoost_features = features_dataframe

        # Create features for ANN
        self.ANN_features = features_dataframe[["Population", "Temperature", "CPI", "GDP"]].values

    def get_prediction(self):
        
        # XG_Boost prediction
        self.XGBoost_prediction = self.XGBoost_model.predict(self.XGBoost_features)
        
        # ANN prediction
        self.ANN_prediction     = self.ANN_model.predict(self.ANN_scaler.transform(self.ANN_features))

    def plotting_data(self):
        
        self.plotting_value = pd.DataFrame([self.Date, self.LSTM_forecast, self.XGBoost_prediction, self.ANN_prediction, self.ARIMA_forecast]).T
        # self.energy_reserve_20 = 42909.9#self.LSTM_forecast.mean() * 1.20
        # self.energy_reserve_35 = 49144.8#self.LSTM_forecast.mean() * 1.35
        # self.energy_reserve_50 = self.LSTM_forecast.mean() * 1.50
        self.plotting_value.columns = ["Date", "LSTM", "XGBoost", "ANN", "ARIMA"]
        self.plotting_value["ANN"] = self.plotting_value["ANN"].abs()

        # Convert Datatype
        #self.plotting_value["Date"] = pd.to_datetime(df['Date'])
        self.plotting_value[["LSTM", "XGBoost", "ANN", "ARIMA"]] = \
        self.plotting_value[["LSTM", "XGBoost", "ANN", "ARIMA"]].astype(float)

        # Insert current maximum capacity.
        self.plotting_value.insert(loc=len(self.plotting_value.columns), 
                                   column="Maximum_capacity",
                                   value= 49144.80
                                   )
        
        self.plotting_value.insert(loc=len(self.plotting_value.columns), 
                                   column="Maximum_capacity_wo_import",
                                   value= 42909.9
                                   )


In [40]:
test2 = forecasting()
test2.plotting_value.head()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator MLPRegressor from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



,Date,LSTM,XGBoost,ANN,ARIMA,Maximum_capacity,Maximum_capacity_wo_import
0,2023-01,27183.696971,26686.539062,35381.756446,26352.073378,49144.8,42909.9
1,2023-02,28179.928109,25581.234375,26916.049887,27158.150121,49144.8,42909.9
2,2023-03,29374.607276,26586.275391,33862.316348,28406.045239,49144.8,42909.9
3,2023-04,31313.368093,26659.753906,36244.405549,29210.645741,49144.8,42909.9
4,2023-05,29411.088627,26550.347656,34979.038026,29151.761566,49144.8,42909.9


In [41]:
test2.plotting_value.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        120 non-null    object 
 1   LSTM                        120 non-null    float64
 2   XGBoost                     120 non-null    float64
 3   ANN                         120 non-null    float64
 4   ARIMA                       120 non-null    float64
 5   Maximum_capacity            120 non-null    float64
 6   Maximum_capacity_wo_import  120 non-null    float64
dtypes: float64(6), object(1)
memory usage: 6.7+ KB


In [42]:
import plotly.express as px

#df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(test2.plotting_value, x="Date", y=test2.plotting_value.columns[1:])
fig.show()
